# Gestore delle Spese Domestiche

## Contesto del Progetto
In un'epoca in cui il controllo delle spese personali e familiari è diventato cruciale per una gestione finanziaria sostenibile, un'applicazione semplice ma efficace può rappresentare un importante valore aggiunto per i consumatori. Il progetto di un gestore delle spese domestiche mira a fornire uno strumento utile e immediato per il monitoraggio delle transazioni economiche quotidiane, con un'interfaccia utente chiara e funzionale.

## Obiettivo del Progetto
L'obiettivo è sviluppare un'applicazione in Python che consenta agli utenti di:
- Tracciare le loro spese quotidiane.
- Generare report mensili delle transazioni.
- Identificare le spese più significative.

Questo strumento fornirà una panoramica finanziaria trasparente, aiutando gli utenti a prendere decisioni consapevoli per migliorare la gestione del budget familiare.

## Funzionalità Chiave
1. **Aggiunta di una transazione**:
   - Gli utenti possono registrare ogni spesa indicando **data**, **descrizione** e **importo**.

2. **Report Mensile**:
   - Generazione di un resoconto che raggruppa le spese per mese, offrendo una visione dettagliata delle transazioni effettuate.

3. **Top 10 Transazioni**:
   - Visualizzazione delle 10 spese più rilevanti, per identificare facilmente le categorie più costose.


---

## IMPLEMENTAZIONE

Andiamo a definire innanzitutto un oggetto di logging utile per il debugging dell'esecuzione del codice:

In [ ]:
from datetime import datetime
import csv
import os
import logging as log


# Configuriamo il logger
logging = log.getLogger()
logging.setLevel(log.DEBUG) # Imposto il livello minimo del logger

#Handler per il Terminale
console_handler = log.StreamHandler()
console_handler.setLevel(log.DEBUG)

#Handler per il File
file_handler = log.FileHandler("app.log")
file_handler.setLevel(log.DEBUG)

#Formatter
formatter = log.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

#Aggiungiamo i 2 Handler al logging
logging.addHandler(console_handler)
logging.addHandler(file_handler)

Ora che abbiamo l'oggetto logging istanziato procediamo con la definizione di una classe secondo paradigma OOP della singola Transazione:

In [ ]:
class Transaction:
    """ Classe che identifica l'oggetto di transazione del progetto """

    def __init__(self, data:str, descrizione:str, importo:str):
        self.data = self._valida_data(data)
        self.importo = self._valida_importo(importo)
        self.descrizione = self._valida_descrizione(descrizione)
        self.filename = './db.csv'




    @staticmethod
    def _valida_data(data) -> datetime :
        """
        Metodo statico per validare una stringa di data e convertirla in un oggetto `datetime`.

        :param data: str - La data in formato 'gg/mm/aaaa'.
        :return: datetime - Oggetto datetime rappresentante la data.
        :raises ValueError: Se la data non è valida.
        """
        try:
            # Parse della stringa nel formato richiesto
            logging.info(f"[Transaction] - Validazione data: {data} avvenuta correttamente")
            return datetime.strptime(data, "%d/%m/%Y")
        except ValueError:
            logging.error(f"[Transaction] - La data '{data}' non è valida. Usa il formato 'gg/mm/aaaa'.")
            raise ValueError(f"La data '{data}' non è valida. Usa il formato 'gg/mm/aaaa'.")




    @staticmethod
    def _valida_importo(importo) -> float:
        """
        Metodo statico per validare l'importo e convertirlo in un float.

        :importo_in: Importo in input
        :return: Float Importo
        :raises ValueError: Se l'importo non è convertibile in float
        """
        try:
            importo_flt = float(importo)
            if (importo_flt <= 0):
                logging.error(f"[Transaction] - L'importo deve essere maggiore di zero.")
                raise ValueError(f"L'importo deve essere maggiore di zero.")
            else:
                logging.info(f"[Transaction] - L'importo : '{importo}' ha superato la validazione.")
                return importo_flt
        except ValueError:
            logging.error(f"[Transaction] - L'importo '{importo}' non è un numero valido. Verifica il carattere separatore che sia il .")
            raise ValueError(f"L'importo '{importo}' non è un numero valido. Verifica il carattere separatore che sia il .")




    @staticmethod
    def _valida_descrizione(descrizione) -> str:
        """
        Metodo statico per validare la descrizione che non sia vuota e non superi i 100 caratteri.

        :descrizione: Descrizione in input
        :return: La stringa "Descrizione" verificata eliminando eventuali spazi bianchi superflui all'inizio e alla fine
        :raises ValueError: Se la descrizione non soddisfa il criterio.
        """
        if len(descrizione)>100:
            logging.error(f"[Transaction] - La Descrizione '{descrizione}' supera i 100 caratteri consentiti.")
            raise ValueError(f"La Descrizione '{descrizione}' supera i 100 caratteri consentiti.")
        if descrizione=="":
            logging.error(f"[Transaction] - La Descrizione '{descrizione}' non puo essere vuota.")
            raise ValueError(f"La Descrizione non puo essere vuota.")

        logging.info(f"[Transaction] - La Descrizione '{descrizione}' ha superato la validazione")
        return descrizione.strip()




    def scrivimi_su_csv(self) -> None : #In questo caso il metodo non è privato ma pubblico quindi elimino il prefisso "_"
        riga = [self.data,self.descrizione, self.importo]

        try:
            #Scrittura dell'intestazione potenziale
            if not os.path.exists(self.filename):
                with open(self.filename, mode='w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file) # Istanzio l'oggetto writer su file
                    writer.writerow(["Data","Descrizione","Importo"])
                    logging.info("[Transaction] - Intestazione del DB scritta su file .csv")

            #Scrittura su file dei dati
            with open(self.filename, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file) # Istanzio l'oggetto writer su file
                writer.writerow(riga)
                logging.info(f"[Transaction] - La transazione del {str(self.data.strftime('%d/%m/%Y'))} e con importo {str(self.importo)} è stata scritta sul DB.")
        except Exception as e:
            logging.error(f"[Transaction] - Errore generico : {str(e)}")



<br><br>Per agevolare l'esecuzione del programma prevediamo l'istanziazione di un oggetto che eseguirà le operazioni sugli oggetti di transazioni e richieste dal progetto e lo chiameremo "TransactionManager":<br><br>

In [ ]:



class TransactionManager:
    """ Classe che gestisce l'insieme delle transazioni, genera report mensili e ottiene le top 10 transazioni """
    def __init__(self):
        self.elenco_transazioni = []
        self.filename = './db.csv'
        self.report = {}

    def aggiorna_elenco_transazioni(self) -> None:

        self.elenco_transazioni.clear()
        # Inizializzo leggendo da file se esiste
        if os.path.exists(self.filename):
            with open(self.filename, mode='r', encoding='utf-8') as file:
                reader = csv.DictReader(file) #Ogni riga è un dizionario e con esso la prima riga è considerata come intestazione per identificare le key del dict.
                for riga in reader:
                    self.elenco_transazioni.append(riga)

        logging.info("[TransactionManager] - Aggiornamento elenco_transazioni effettuato.")



    def add_transaction(self) -> None :
        """
        Metodo che aggiunge la singola transazione alla lista transazioni salvandola sul DB in csv ed aggiorna la lista interna alla classe.
        """

        try:
            data_tr = input("Inserisci Data Transazione:")
            importo_tr = input("Inserisci Importo Transazione:")
            descr_tr = input("Inserisci Descrizione Transazione:")
            tr = Transaction(data_tr, descr_tr, importo_tr)
            tr.scrivimi_su_csv()
            logging.info("[TransactionManager] - Operazione di aggiunta della Transazione eseguita con successo.")
            self.aggiorna_elenco_transazioni()

        except Exception as e:
            logging.error(f"[TransactionManager] - Errore generico: {str(e)}")




    def genera_report(self) -> None:
        """
        Metodo che alimenta il report Anno_mese (dizionario) aggregando i dati giornalieri.
        """

        self.aggiorna_elenco_transazioni()
        elenco_anno_mese = []
        self.report = {}

        #Aggiungiamo un campo 'Anno-Mese
        for elem in self.elenco_transazioni:
            anno_mese = str(datetime.strptime(elem['Data'],'%Y-%m-%d %H:%M:%S').year) + '_' + str(datetime.strptime(elem['Data'],'%Y-%m-%d %H:%M:%S').month)
            if anno_mese not in elenco_anno_mese:
                elenco_anno_mese.append(anno_mese)

        #Adesso aggrego

        for elem_anno_mese in elenco_anno_mese:
            tot_imp=0
            for transazione in self.elenco_transazioni:
                if elem_anno_mese == str(datetime.strptime(transazione['Data'],'%Y-%m-%d %H:%M:%S').year) + '_' + str(datetime.strptime(transazione['Data'],'%Y-%m-%d %H:%M:%S').month) :
                    tot_imp += float(transazione['Importo'])

            self.report[elem_anno_mese] = tot_imp


        logging.info("[TransactionManager] - Report Anno-Mese generato")




    def stampa_report(self) -> None:
        """
        Metodo che stampa il report Anno_Mese (dizionario).
        """

        if self.report:
            for key, value in self.report.items():
                print(f"{key} - {value}")
        else:
            logging.info(f"[TransactionManager] - Report non generato in quanto vuoto")


    def genera_top_10_transazioni(self) -> None:
        """
        Metodo che stampa le prime 10 transazioni con importi piu alti.
        """

        self.aggiorna_elenco_transazioni() # Vado a generarmi il report per Anno-Mese dal file csv
        elenco_ordinato = sorted(self.elenco_transazioni, key = lambda x: float(x['Importo']), reverse=True)
        count=0
        for elem in elenco_ordinato:
            if count <=9:
                 print(f"{elem['Data']} - {elem['Descrizione']} - {elem['Importo']}")
                 count += 1

        logging.info("[TransactionManager] - Stampa prime 10 transazioni effettuata")




<br>
Infine andiamo a crearci una classe che governa le interazioni dell'utente e quindi verso il TransactionManager:
<br><br>

In [ ]:
import sys

class TransactionManagerUI:

    def __init__(self):
        self.opzione = 0
        self.tr_manager = TransactionManager()



    def get_opzione(self) -> int:
        """
        Metodo che restituisce l'opzione attiva utente
        """
        return self.opzione


    def opzione_non_valida(self) -> None:
        print("Opzione non valida. Riprova.")
        logging.error(f"[TransactionManagerUI] - Opzione non valida selezionata: {self.opzione}")


    def stampa_opzioni(self) -> None:
        """
        Metodo che cicla la stampa a video fin quando l'utente non ha selezionato un valore valido per "opzione"
        """
        self.opzione=0
        try:
            while int(self.opzione) not in (1,2,3,4):
                print("\n\n---Menu------------------------")
                print("1. Aggiungi Transazione")
                print("2. Genera Report Mensile")
                print("3. Top 10 Transazioni")
                print("4. Esci...")
                print("-------------------------------")
                print("\n\n")
                self.opzione = int(input("Inserisci opzione:"))
                logging.info(f"[TransactionManagerUI] - Valore utente è: {self.opzione}")
        except Exception as e:
            logging.debug(f"[TransactionManagerUI] - Errore nell'input inserito")
            raise ValueError(f"[TransactionManagerUI] - Errore nell'input inserito")


    def lancia_add_transazione(self) -> None:
        """
        Metodo che esegue il sub-metodo add_transaction
        """
        self.tr_manager.add_transaction()
        logging.info("[TransactionManagerUI] - esecuzione add_transiction completata")


    def lancia_genera_report(self) -> None:
        """
        Metodo che esegue il sub-metodo stampa_report e genera_report
        """
        self.tr_manager.genera_report()
        logging.info("[TransactionManagerUI] - Esecuzione genera_report completata")
        self.tr_manager.stampa_report()
        logging.info("[TransactionManagerUI] - Esecuzione stampa_report completata")



    def lancia_top_10_tr(self) -> None:
        """
        Metodo che esegue il sub-metodo lancia_top_10_tr
        """
        self.tr_manager.genera_top_10_transazioni()
        logging.info("[TransactionManagerUI] - Esecuzione lancia_transazioni_top_10 completata")



    def esci(self) -> None:
        """
        Metodo che chiude il programma-
        """
        print("Grazie per aver usato il programma. Arrivederci!")
        logging.info("Chiusura del programma richiesta dall'utente.")
        sys.exit(0)



    def esegui(self) -> None:
        """
        Metodo che esegue il sub-metodo stampa_opzioni
        """
        associa = {
            1:self.lancia_add_transazione,
            2:self.lancia_genera_report,
            3:self.lancia_top_10_tr,
            4:self.esci
        }
        self.stampa_opzioni()
        associa.get(self.opzione, self.opzione_non_valida)()
        logging.info("[TransactionManagerUI] - Esecuzione dell'opzione utente completata.")



<br>
Proviamo a eseguire il programma:
<br><br>

In [ ]:

app = TransactionManagerUI()
while(1>0):
    app.esegui()





---Menu------------------------
1. Aggiungi Transazione
2. Genera Report Mensile
3. Top 10 Transazioni
4. Esci...
-------------------------------





Inserisci opzione: 2


2024-12-22 00:33:28,506 - INFO - [TransactionManagerUI] - Valore utente è: 2
2024-12-22 00:33:28,507 - INFO - [TransactionManager] - Aggiornamento elenco_transazioni effettuato.
2024-12-22 00:33:28,509 - INFO - [TransactionManager] - Report Anno-Mese generato
2024-12-22 00:33:28,510 - INFO - [TransactionManagerUI] - Esecuzione genera_report completata
2024-12-22 00:33:28,511 - INFO - [TransactionManagerUI] - Esecuzione stampa_report completata
2024-12-22 00:33:28,511 - INFO - [TransactionManagerUI] - Esecuzione dell'opzione utente completata.


2024_3 - 123445678.0
2025_12 - 1400.0
2021_3 - 4020.0
2022_3 - 222.0
2922_8 - 424242.0
2222_2 - 1414141.0
2050_1 - 202020202.0
2030_6 - 12414141.0
2077_4 - 29292929.0
2099_1 - 9999999999.0
2100_1 - 1e+17


---Menu------------------------
1. Aggiungi Transazione
2. Genera Report Mensile
3. Top 10 Transazioni
4. Esci...
-------------------------------





Inserisci opzione: 3


2024-12-22 00:33:31,369 - INFO - [TransactionManagerUI] - Valore utente è: 3
2024-12-22 00:33:31,370 - INFO - [TransactionManager] - Aggiornamento elenco_transazioni effettuato.
2024-12-22 00:33:31,371 - INFO - [TransactionManager] - Stampa prime 10 transazioni effettuata
2024-12-22 00:33:31,371 - INFO - [TransactionManagerUI] - Esecuzione lancia_transazioni_top_10 completata
2024-12-22 00:33:31,372 - INFO - [TransactionManagerUI] - Esecuzione dell'opzione utente completata.


2100-01-01 00:00:00 - gnwgewngwgnwngwngwnnw - 1e+17
2099-01-01 00:00:00 - ogbwogbw - 9999999999.0
2050-01-01 00:00:00 - qwrqr - 202020202.0
2024-03-03 00:00:00 - rqrqrq - 123445555.0
2077-04-04 00:00:00 - wfiq - 29292929.0
2030-06-03 00:00:00 - dgnogaa - 12414141.0
2222-02-15 00:00:00 - sgasg - 1414141.0
2922-08-03 00:00:00 - wgwgw - 424242.0
2021-03-13 00:00:00 - qwe - 4020.0
2025-12-18 00:00:00 - colazione - 1000.0


---Menu------------------------
1. Aggiungi Transazione
2. Genera Report Mensile
3. Top 10 Transazioni
4. Esci...
-------------------------------



